In [1]:
import numpy as np
import cv2
import os


In [5]:
# 1. Загрузка и сохранение монохромного изображения
IMAGE_PATH = "sar_1_gray.jpg"  
img_gray = cv2.imread(IMAGE_PATH, cv2.IMREAD_GRAYSCALE)

# Сохраняем исходное изображение в текстовый файл 
np.savetxt("image_original.txt", img_gray, fmt="%d")
img = img_gray.astype(np.float32)

In [6]:
# 2. Реализация вейвлет-преобразования Хаара
def haar_transform(image: np.ndarray):
    h, w = image.shape

    # Делаем размеры чётными
    if h % 2 == 1:
        image = image[:-1, :]
        h -= 1
    if w % 2 == 1:
        image = image[:, :-1]
        w -= 1

    # Проход по строкам 
    temp = np.zeros_like(image, dtype=np.float32)
    for i in range(h):
        k = 0
        for j in range(0, w, 2):
            avg  = (image[i, j] + image[i, j + 1]) / 2.0   
            diff = (image[i, j] - image[i, j + 1]) / 2.0  
            temp[i, k] = avg
            temp[i, k + w // 2] = diff
            k += 1

    result = np.zeros_like(temp, dtype=np.float32)
    for j in range(w):
        k = 0
        for i in range(0, h, 2):
            avg  = (temp[i, j] + temp[i + 1, j]) / 2.0    
            diff = (temp[i, j] - temp[i + 1, j]) / 2.0    
            result[k, j] = avg
            result[k + h // 2, j] = diff
            k += 1

    LL = result[:h // 2, :w // 2]  
    LH = result[:h // 2,  w // 2:]  
    HL = result[h // 2:, :w // 2]   
    HH = result[h // 2:,  w // 2:]  

    return LL, LH, HL, HH

LL, LH, HL, HH = haar_transform(img)

In [7]:
# 3. Квантование высокочастотных компонент
N_QUANTS = 4  # клво квантов

def quantize_component(coeffs: np.ndarray, n_quants: int):
    min_val = float(np.min(coeffs))
    max_val = float(np.max(coeffs))

    if max_val == min_val:
        q = np.zeros_like(coeffs, dtype=np.int16)
        step = 1.0
        return q, min_val, step

    step = (max_val - min_val) / n_quants

    q = np.floor((coeffs - min_val) / step).astype(np.int16)

    q = np.clip(q, 0, n_quants - 1)

    return q, min_val, step

LH_q, LH_min, LH_step = quantize_component(LH, N_QUANTS)
HL_q, HL_min, HL_step = quantize_component(HL, N_QUANTS)
HH_q, HH_min, HH_step = quantize_component(HH, N_QUANTS)

In [9]:
# 4. RLE-кодирование высокочастотных компонент
def run_length_encode(arr: np.ndarray):
    flat = arr.flatten()
    encoded = []

    if flat.size == 0:
        return encoded

    current_value = int(flat[0])
    count = 1

    for v in flat[1:]:
        v = int(v)
        if v == current_value:
            count += 1
        else:
            encoded.append((current_value, count))
            current_value = v
            count = 1

    encoded.append((current_value, count))

    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

In [10]:
# 5. Сохранение результата и сравнение результатов
OUTPUT_PATH = "wavelet_rle.txt"

with open("wavelet_rle.txt", "w") as f:
    np.savetxt(f, LL, fmt="%.2f")
    f.write("\n")
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    f.write("\n")
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")
    f.write("\n")
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")